In [2]:
import numpy as np
import cv2

img = cv2.imread('lena.bmp', 0)
rows, cols = img.shape

cal = np.zeros(256)

for i in range(rows):
    for j in range(cols):
        cal[img[i][j]] += 1
        if img[i][j] > 127:
            img[i][j] = 255
        else:
            img[i][j] = 0

cv2.imwrite('binary.jpg', img)
print img


div_scalar = np.amax(cal) / (256 * 0.9) # max value is about 0.9 position from bottom
his = np.zeros((256, 256))

for i in range(256):
    for j in range(256):
        if i > 255 - int(cal[j] / div_scalar):
            his[i][j] = 255

cv2.imwrite('histogram.jpg', his)

img_cp = np.zeros((rows+1, cols+1), dtype=int)
img_cp[1:, 1:] = img
label = 0
edges = set()
# first pass
for i in range(1, rows + 1, 1):
    for j in range(1, cols + 1, 1):
        if img_cp[i][j] == 0:
            continue
            
        left, up = img_cp[i][j-1], img_cp[i-1][j]
        if left == 0:
            if up == 0:
                label += 1
                img_cp[i][j] = label
            else:
                img_cp[i][j] = up
        else:
            if up == 0:
                img_cp[i][j] = left
            else:
                if left > up: 
                    img_cp[i][j] = up
                    edges.add((left, up))
                elif up > left:
                    img_cp[i][j] = left
                    edges.add((up, left))
                else:
                    img_cp[i][j] = left
            
label_pt = np.arange(label + 1, dtype=int)

def Find(index):
    if label_pt[index] != index:
        label_pt[index] = Find(label_pt[index])
    return label_pt[index]

def Union(x, y):
    xRoot = Find(x)
    yRoot = Find(y)
    
    if xRoot == yRoot:         
       return
    elif xRoot < yRoot:
        label_pt[yRoot] = xRoot
    else:
        label_pt[xRoot] = yRoot

for edge in edges:
    x, y = edge
    Union(x, y)
    
label_count = np.zeros(label + 1, dtype=int)
# second pass
for i in range(1, rows + 1, 1):
    for j in range(1, cols + 1, 1):
        if img_cp[i][j] == 0:
            continue
            
        img_cp[i][j] = Find(label_pt[img_cp[i][j]])
        label_count[img_cp[i][j]] += 1  
        
rbg_img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
img_cp = img_cp[1:, 1:]
for cnt in range(label+1):
    if label_count[cnt] >= 500:
        min_i, min_j = rows, cols
        max_i, max_j = 0, 0
        for i in range(rows):
            for j in range(cols):
                if img_cp[i][j] == cnt:
                    if i < min_i:
                        min_i = i
                    elif i > max_i:
                        max_i = i
                    if j < min_j:
                        min_j = j
                    elif j > max_j:
                        max_j = j
                    
        cv2.rectangle(rbg_img, (min_j, min_i), (max_j, max_i), (0, 0, 255), 1)
        
cv2.imwrite('connected_components.jpg', rbg_img)


[[255 255 255 ..., 255 255 255]
 [255 255 255 ..., 255 255 255]
 [255 255 255 ..., 255 255 255]
 ..., 
 [  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]]


True

In [5]:
np.set_printoptions(threshold=10)

print (cv2.imread('binary.jpg'))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

